In [ ]:
import sys
sys.path.append('../')
import pandas as pd


from dynalign.experiments.defaults import DATASETS, L2_ALIGNERS
from dynalign.experiments.parsers import MetricsParser

In [ ]:
mp = MetricsParser(aligners=L2_ALIGNERS)
df = pd.DataFrame(mp.parse())

In [ ]:
ds_ratios = {}

for dataset in DATASETS:

    ds_df = df[(df.dataset == dataset)]

    n2v_loss_fitered_df = ds_df[
        (ds_df.metric == "n2v_loss")
        & (ds_df.epoch == ds_df.epoch.max())
        & (ds_df.snapshot < ds_df.snapshot.max())
    ]

    l2_loss_fitered_df = ds_df[
        (ds_df.metric == "alignment_loss")
        & (ds_df.epoch == ds_df.epoch.max())
        & (ds_df.snapshot < ds_df.snapshot.max())
    ]

    losses_ratio = n2v_loss_fitered_df.groupby(["snapshot", "epoch"]).mean().drop(
        "run", axis=1
    ) / l2_loss_fitered_df.groupby(["snapshot", "epoch"]).mean().drop("run", axis=1)

    ds_ratios[dataset] = {
        "mean": losses_ratio.dropna().mean().item(),
        "std": losses_ratio.dropna().std().item(),
    }

In [ ]:
pd.DataFrame.from_dict(ds_ratios)